In [35]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from time import time

import torch
import torch.utils.data as data

import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
### Configurations
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/train.csv'

In [37]:
mean, std = (0.5, 0.5, 0.5), (0.2, 0.2, 0.2)

In [38]:
from albumentations import *
from albumentations.pytorch import ToTensorV2


def get_transforms(need=('train', 'val'), img_size=(512, 384), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
    """
    train 혹은 validation의 augmentation 함수를 정의합니다. train은 데이터에 많은 변형을 주어야하지만, validation에는 최소한의 전처리만 주어져야합니다.
    
    Args:
        need: 'train', 혹은 'val' 혹은 둘 다에 대한 augmentation 함수를 얻을 건지에 대한 옵션입니다.
        img_size: Augmentation 이후 얻을 이미지 사이즈입니다.
        mean: 이미지를 Normalize할 때 사용될 RGB 평균값입니다.
        std: 이미지를 Normalize할 때 사용될 RGB 표준편차입니다.

    Returns:
        transformations: Augmentation 함수들이 저장된 dictionary 입니다. transformations['train']은 train 데이터에 대한 augmentation 함수가 있습니다.
    """
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            HorizontalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            GaussNoise(p=0.5),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            Resize(img_size[0], img_size[1]),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [39]:
### 마스크 여부, 성별, 나이를 mapping할 클래스를 생성합니다.

class MaskLabels:
    mask = 0
    incorrect = 1
    normal = 2

class GenderLabels:
    male = 0
    female = 1

class AgeGroup:
    map_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 60 else 2

In [40]:
class MaskBaseDataset(data.Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1.jpg": MaskLabels.mask,
        "mask2.jpg": MaskLabels.mask,
        "mask3.jpg": MaskLabels.mask,
        "mask4.jpg": MaskLabels.mask,
        "mask5.jpg": MaskLabels.mask,
        "incorrect_mask.jpg": MaskLabels.incorrect,
        "normal.jpg": MaskLabels.normal
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, img_dir, transform=None):
        """
        MaskBaseDataset을 initialize 합니다.

        Args:
            img_dir: 학습 이미지 폴더의 root directory 입니다.
            transform: Augmentation을 하는 함수입니다.
        """
        self.img_dir = img_dir
        self.mean = mean
        self.std = std
        self.transform = transform

        self.setup()

    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform
        
    def setup(self):
        """
        image의 경로와 각 이미지들의 label을 계산하여 저장해두는 함수입니다.
        """
        profiles = os.listdir(self.img_dir)
        for profile in profiles:
            for file_name, label in self._file_names.items():
                img_path = os.path.join(self.img_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                if os.path.exists(img_path):
                    self.image_paths.append(img_path)
                    self.mask_labels.append(label)

                    id, gender, race, age = profile.split("_")
                    gender_label = getattr(GenderLabels, gender)
                    age_label = AgeGroup.map_label(age)

                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)

    def __getitem__(self, index):
        """
        데이터를 불러오는 함수입니다. 
        데이터셋 class에 데이터 정보가 저장되어 있고, index를 통해 해당 위치에 있는 데이터 정보를 불러옵니다.
        
        Args:
            index: 불러올 데이터의 인덱스값입니다.
        """
        # 이미지를 불러옵니다.
        image_path = self.image_paths[index]
        image = Image.open(image_path)
        
        # 레이블을 불러옵니다.
        mask_label = self.mask_labels[index]
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]
        multi_class_label = mask_label * 6 + gender_label * 3 + age_label
        
        # 이미지를 Augmentation 시킵니다.
        image_transform = self.transform(image=np.array(image))['image']
        return image_transform, multi_class_label

    def __len__(self):
        return len(self.image_paths)

In [41]:
# 정의한 Augmentation 함수와 Dataset 클래스 객체를 생성합니다.
transform = get_transforms(mean=mean, std=std)

dataset = MaskBaseDataset(
    img_dir=img_dir
)

# train dataset과 validation dataset을 8:2 비율로 나눕니다.
n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])

# 각 dataset에 augmentation 함수를 설정합니다.
train_dataset.dataset.set_transform(transform['train'])
val_dataset.dataset.set_transform(transform['val'])

In [42]:
# training dataloader은 데이터를 섞어주어야 합니다. (shuffle=True)
train_dataloader = data.DataLoader(
    train_dataset,
    batch_size=12,
    num_workers=4,
    shuffle=True
)

val_dataloader = data.DataLoader(
    val_dataset,
    batch_size=12,
    num_workers=4,
    shuffle=False
)

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#from dataset.dataloader import getDataloader
import torchvision
#from model.resnet import ResNet
from tqdm.notebook import tqdm

device = 'cuda'

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(in_features=512, out_features=18, bias=True)

#model = torchvision.models.vgg16(pretrained=True)
#model.classifier[6] = nn.Linear(in_features=4096, out_features=18, bias=True)

print(model)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [46]:
epochs = 50
min_val_loss = 999
min_val_acc = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_acc = 0
    print("----------------Train!----------------")
    for batch_iter, batch in enumerate(tqdm(train_dataloader)):
        
        batch_in, batch_out = batch
        batch_in = batch_in.to(device)
        batch_out = batch_out.to(device)

        # Forward path

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            y_pred = model(batch_in)
            loss_out = criterion(y_pred, batch_out)
            acc_out = torch.sum(torch.max(y_pred, 1)[1] == batch_out.data)/len(batch_out.data)
            if not batch_iter%100 : 
                print('[epoch: {}, iter: {}] train acc : {}, train loss: {}'.format(epoch, batch_iter, acc_out, loss_out))

        # backpropagate
        loss_out.backward()

        # optimizer update
        optimizer.step()
        total_loss += loss_out
        total_acc += acc_out
    avg_loss = total_loss/len(train_dataloader)
    avg_acc = total_acc/len(train_dataloader)
    print('[epoch: {}] average train acc: {} average train loss: {}'.format(epoch, avg_acc, avg_loss))

    print("----------------validation!----------------")
    with torch.no_grad():
        model.eval() # evaluate (affects DropOut and BN)
        total_val_loss = 0
        total_val_acc = 0
        for val_iter, val_batch in enumerate(tqdm(val_dataloader)):
            val_batch_in, val_batch_out = val_batch
            val_batch_in = val_batch_in.to(device)
            val_batch_out = val_batch_out.to(device)
            y_pred = model(val_batch_in)
            loss_out = criterion(y_pred, val_batch_out)
            acc_out = torch.sum(torch.max(y_pred, 1)[1] == val_batch_out.data)/len(val_batch_out.data)
            if not val_iter%10 :
                print('[epoch: {}, iter: {}] validation acc: {} validation loss: {}'.format(epoch, val_iter, acc_out, loss_out))
            total_val_loss += loss_out
            total_val_acc += acc_out
        avg_val_loss = total_val_loss/len(val_dataloader)
        avg_val_acc = total_val_acc/len(val_dataloader)
        
    print ("epoch:[%d] val_acc:[%.3f] val_loss:[%.3f]."%(epoch, avg_val_acc, avg_val_loss))

    if avg_val_loss < min_val_loss:
        torch.save(model, '/opt/ml/checkpoint/{}_{}_{}.pt'.format('resnet18', epoch, avg_val_loss))
        min_val_loss = avg_val_loss
        print('----------------model saved!----------------')

----------------Train!----------------


[epoch: 0, iter: 0] train acc : 0.0833333358168602, train loss: 2.8032283782958984
[epoch: 0, iter: 100] train acc : 0.9166666865348816, train loss: 0.5981107354164124
[epoch: 0, iter: 200] train acc : 1.0, train loss: 0.3879980742931366
[epoch: 0, iter: 300] train acc : 0.75, train loss: 0.7663399577140808



KeyboardInterrupt: 

In [ ]:
#albumentation을 이용해 테스트 데이터셋 코드를 위의 밸리데이션 데이터셋 코드와 최대한 같게 한다면 더 좋은 테스트 결과가 나올 것 같습니다.  

import os
from torchvision.transforms import Resize, ToTensor, Normalize
from PIL import Image
from torchvision import transforms

class TestDataset(data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

test_dir = '/opt/ml/input/data/eval'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
'''
transform = Compose([
            Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
        '''
transform = transforms.Compose([
            Resize((512, 384), Image.BILINEAR),
            ToTensor(),
            Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)),
        ])
dataset = TestDataset(image_paths, transform)

loader = data.DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = torch.load('/opt/ml/checkpoint/resnet18_1_0.16431666910648346.pt')
model.eval()

#pred = model(dataset[0])
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join('/opt/ml/submissions', 'submission_resnet3.csv'), index=False)
print('test inference is done!')